# Towards Evaluating Creativity in Language

In [73]:
# Imports...
import torch
import matplotlib.pyplot
import numpy

# handy features for downloading the datasets
import pathlib, requests

import nltk
# nltk.download()

from datasets import load_dataset

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Load datasets

In [56]:
# Check if the file exists, otherwise download it

cloze_train_path = pathlib.Path("./data/cloze_test_2018_train.csv")
if not cloze_train_path.exists():
    cloze_train_path.write_bytes(requests.get("https://goo.gl/0OYkPK").content)

cloze_val_path = pathlib.Path("./data/cloze_test_2018_val.csv")
if not cloze_val_path.exists():
    cloze_val_path.write_bytes(requests.get("https://goo.gl/XWjas1").content)

cloze_test_path = pathlib.Path("./data/cloze_test_2018_test.csv")
if not cloze_test_path.exists():
    cloze_test_path.write_bytes(requests.get("https://goo.gl/BcTtB4").content)

# print(cloze_test_path.__str__())

val_ds = load_dataset('csv', "val", data_files=cloze_val_path.__str__())
tiny_shakespeare = load_dataset('tiny_shakespeare')

# Load the datasets later
# train_ds = load_dataset('csv', "train", data_files=cloze_train_path.__str__())
# test_ds = load_dataset('csv', "test", data_files=cloze_test_path.__str__())

Using custom data configuration val-85d52daad032b236
Found cached dataset csv (/home/matey/.cache/huggingface/datasets/csv/val-85d52daad032b236/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)



100%|██████████| 1/1 [00:00<00:00, 328.35it/s]











Dataset tiny_shakespeare downloaded and prepared to /home/matey/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e. Subsequent calls will reuse this data.





100%|██████████| 3/3 [00:00<00:00, 528.69it/s]


In [74]:
# Yikes, the chaining to access this
# tiny_shakespeare["train"][0]["text"].__len__()

nltk.pos_tag(nltk.word_tokenize("The quick brown fox jumps over the lazy dog."))

[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumps', 'VBZ'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog', 'NN'),
 ('.', '.')]

### Potential creativity measures
#### Usage of more uncommon words
**Example:**
- "The quick brown fox jumps over the lazy dog."
- "The swift hazel-furred fox leaps over the idle dog."
**Idea:**
Adapt resources such as WordNet for finding semantically similar words and compare them to their most used synonym. The more uncommon the word, the more creative the sentence.
Potential problems: words could be too far off from their actual meaning in the context. For example, here "leaps" is a synonym for "jumps", but using the word "vaults" or "springs" might not fit the context.



In [ ]:
# Idea: 